In [1]:
from scipy.io import arff
import pandas as pd
import os
from sklearn import tree
from sklearn.metrics import accuracy_score, mean_absolute_error,mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt
import pydot
import numpy as np

In [2]:
results = pd.DataFrame()
results_columns = pd.DataFrame()

for dirpath, dirnames, filenames in os.walk("../data_sets/data/", topdown=False):
    for name in filenames:
        fullpath = os.path.join(dirpath, name)
        data = pd.DataFrame(arff.loadarff(fullpath)[0])
        data['class'] = data['class'].str.decode('utf-8')
        data['year'] = float(name[0])
        results = pd.concat([results,data],axis=0).reset_index(drop=True)
        
                

In [3]:
#sort by year
results = results.sort_values(by=['year'])

In [4]:
results.head()

,Attr1,Attr2,Attr3,Attr4,Attr5,Attr6,Attr7,Attr8,Attr9,Attr10,...,Attr57,Attr58,Attr59,Attr60,Attr61,Attr62,Attr63,Attr64,class,year
43404,0.014946,0.946480,0.03211,1.03630,-20.5810,0.000000,0.015260,0.056357,2.96940,0.053341,...,0.28021,0.97443,1.179200,15.0360,4.1741,108.640,3.3599,35.11800,1,1.0
38710,-0.083327,0.942880,-0.17625,0.69310,-92.9120,-0.095666,-0.078056,-0.005651,0.87141,-0.005328,...,15.64000,1.14760,-69.182000,18.4010,2.1509,278.570,1.3103,1.25010,0,1.0
38711,0.161240,0.657540,0.15056,1.45130,-9.5086,0.498510,0.206490,0.498930,1.10840,0.328070,...,0.49147,0.90223,0.987410,10.7320,10.0460,56.648,6.4433,4.16700,0,1.0
38712,0.031386,0.038397,0.19497,6.12450,358.7300,0.010658,0.031643,25.044000,1.18860,0.961600,...,0.03264,0.84130,0.000365,5.3563,2.8605,66.169,5.5162,0.27362,0,1.0
38713,-0.092135,1.132700,-0.19627,0.82672,-99.4520,-0.140320,-0.092135,-0.117120,0.95725,-0.132660,...,0.69452,1.04470,0.000000,5.9660,2.9435,252.190,1.4473,25.77400,0,1.0


In [ ]:
results.info(verbose=True)

In [ ]:
missing_cols = results.columns[results.isnull().any()]

In [ ]:
results.describe()

In [ ]:

plt.figure(figsize=(10,10));
sns.displot(
    data=results[missing_cols].isna().melt(value_name="missing"),
    y="variable",
    hue="missing",
    multiple="fill"
);

***
# Impute missing data

In [ ]:
mean_imp = SimpleImputer(missing_values = pd.NA,strategy='mean')
median_imp = SimpleImputer(missing_values = pd.NA,strategy = 'median')
freq_imp = SimpleImputer(missing_values = pd.NA,strategy = 'most_frequent')

In [ ]:
df_impute = results


df_mean = pd.DataFrame(mean_imp.fit_transform(df_impute),columns = df_impute.columns)
df_median = pd.DataFrame(median_imp.fit_transform(df_impute),columns = df_impute.columns)
df_freq = pd.DataFrame(freq_imp.fit_transform(df_impute),columns = df_impute.columns)

# Begin Modeling

In [ ]:

X = df_mean.loc[:, df_mean.columns != 'class']
y = df_mean['class']
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    shuffle=False,
                                                    random_state=0)

In [ ]:
model = RandomForestClassifier()

# Train the model on training data
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
model_results=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
model_results

In [ ]:
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred)))

In [ ]:

accuracy = accuracy_score(y_test,y_pred)
print('Accuracy:', round(accuracy, 2), '%.')

In [ ]:

plt.figure(figsize=(5, 7))


ax = sns.distplot(y, hist=False, color="r", label="Actual Value")
sns.distplot(y_pred, hist=False, color="b", label="Fitted Values" , ax=ax)


plt.title('Actual vs Fitted Values')


plt.show()
plt.close()

In [ ]:

# Pull out one tree from the forest
Tree =  model.estimators_[5]
# Export the image to a dot file

plt.figure(figsize=(25,15))
tree.plot_tree(Tree,filled=True, 
              rounded=True, 
              fontsize=14);